In [15]:
import mlflow
import cloudpickle
import os

# Define the path to save the model
model_path = "./model/model.pkl"

# Create the directory if it does not exist
os.makedirs(os.path.dirname(model_path), exist_ok=True)

# Set the MLflow tracking URI
mlflow.set_tracking_uri("http://seito.lavbic.net:5000")

# Load the model from MLflow registry
model_uri = "models:/Race prediction@production"
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Create a simple wrapper class that can be pickled
class ModelWrapper:
	def __init__(self, mlflow_model):
		self.mlflow_model = mlflow_model
		
	def predict(self, X):
		return self.mlflow_model.predict(X)

# Create wrapper instance
wrapper = ModelWrapper(loaded_model)

# Save the wrapper using cloudpickle which handles more complex objects
print(f"Saving model wrapper to '{model_path}'...")
with open(model_path, 'wb') as f:
	cloudpickle.dump(wrapper, f)
print("Model wrapper saved successfully.")

Saving model wrapper to './model/model.pkl'...
Model wrapper saved successfully.


In [2]:
import requests
import json

# URL of the Flask server (adjust the port if necessary)
url = "http://seito.lavbic.net:6000/predict"

# Index of the race data you want to predict
payload = {
    "index": 0  # Replace with the actual index you want to use
}

# Send the POST request
try:
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(payload), timeout=60)
    
    # Check if the response is successful
    if response.status_code == 200:
        result = response.json()
        print("Prediction result:", result)
    else:
        print("Error:", response.text)
except requests.exceptions.RequestException as e:
    print("Request failed:", e)


Error: {"error":"Failed to enforce schema of data '[[ 8.29857663e-01  4.74131719e-01  5.71027792e-01 ...  2.70000000e+01\n   1.12000000e+02  2.39800000e+03]\n [ 8.29857663e-01  4.74131719e-01  5.71027792e-01 ...  6.10000000e+01\n   2.46000000e+02  1.56700000e+03]\n [ 8.29857663e-01  4.74131719e-01  5.71027792e-01 ...  7.00000000e+00\n  -1.00000000e+00  2.41100000e+03]\n ...\n [ 8.29857663e-01  4.74131719e-01  5.71027792e-01 ...  0.00000000e+00\n   0.00000000e+00  0.00000000e+00]\n [ 8.29857663e-01  4.74131719e-01  5.71027792e-01 ...  0.00000000e+00\n   0.00000000e+00  0.00000000e+00]\n [ 8.29857663e-01  4.74131719e-01  5.71027792e-01 ...  0.00000000e+00\n   0.00000000e+00  0.00000000e+00]]' with schema '[Tensor('float32', (-1, 227))]'. Error: dtype of input float64 does not match expected dtype float32"}

